In [ ]:
%pip install semantic-link

# List Datasets

In [ ]:
import sempy.fabric as fabric
dataset_list = fabric.list_datasets()

dataset_list

# Read Tables

In [ ]:
dataset = "USAShips"

table_list = fabric.list_tables(dataset)

table_list

In [ ]:
table_columns = fabric.list_columns(dataset)

table_columns

In [ ]:

shipment_df = fabric.read_table(dataset, 'Shipment')

shipment_df

In [ ]:
shipment_df.to_lakehouse_table(name="shipment",mode="overwrite")

# Relationships

In [ ]:
from sempy.relationships import plot_relationship_metadata
from sempy.relationships import find_relationships
from sempy.fabric import list_relationship_violations

relationships = fabric.list_relationships(dataset)
relationships

In [ ]:
plot_relationship_metadata(relationships)

### Relationships Violations

In [ ]:
tables = {table: fabric.read_table(dataset, table) for table in fabric.list_tables(dataset)['Name']}

tables.keys()

In [ ]:
list_relationship_violations(tables, fabric.list_relationships(dataset))

### Find New Relationships

In [ ]:
dataset_no_relationship = "USAShips_NoRelations"
actual_relationships = fabric.list_relationships(dataset_no_relationship)
plot_relationship_metadata(actual_relationships)

In [ ]:
tables = {table: fabric.read_table(dataset_no_relationship, table) for table in fabric.list_tables(dataset_no_relationship)['Name']}

tables.keys()

In [ ]:

additional_relationships = find_relationships(
    tables,
    exclude=actual_relationships,
    name_similarity_threshold=0.7, # Default=0.8
    coverage_threshold=0.7 # Default=1 
)

additional_relationships

# Dependencies

In [ ]:
from sempy.dependencies import plot_dependency_metadata

region = fabric.read_table(dataset, "Region")
region

### Find Dependencies

In [ ]:
dependencies = region.find_dependencies()
dependencies

In [ ]:
plot_dependency_metadata(dependencies)

### Find Dependencies Violations

In [ ]:
region.plot_dependency_violations('Zip_code', 'State')

In [ ]:
region.plot_dependency_violations('Zip_code', 'City')

In [ ]:
region.list_dependency_violations('Zip_code', 'City')

# Measures

### List measures

In [ ]:
dataset_flights = "SL_Flights"

fabric.list_measures(dataset_flights)

### Evaluate Measure

In [ ]:
#fabric.evaluate_measure(dataset_flights, measure="# Delays")

# Delays by Airline and Airline Country
fabric.evaluate_measure(dataset_flights, measure="# Delays", groupby_columns=["Airline[Airline]", "Airline[Country (Airline)]"])


In [ ]:
# Delays by Airline and Origin Airport only from Alaska

fabric.evaluate_measure(dataset_flights, measure="# Delays", groupby_columns=["Airline[Airline]", "Airport (Origin)[Airport (Origin)]"], filters={"Airport (Origin)[State (Origin)]": ["Alaska"]})

In [ ]:
%load_ext sempy

In [ ]:
%%dax "SL_Flights"

SELECT * FROM $System.DISCOVER_CALC_DEPENDENCY WHERE OBJECT_TYPE = 'ACTIVE_RELATIONSHIP'

In [ ]:
spark.conf.set("spark.sql.catalog.pbi", "com.microsoft.azure.synapse.ml.powerbi.PowerBICatalog")

In [ ]:
%%sql 

SELECT
    
    `Airport (Origin)[Airport (Origin)]` AS Origin,
    SUM(`# Airlines`) as TotalAirlines

FROM pbi.`SL_Flights`.`_Metrics`
GROUP BY `Airport (Origin)[Airport (Origin)]`